In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seawater as sw
import cartopy.crs as ccrs                   # import projections
import cartopy.feature as cf                 # import features
import uuid
from scipy import interpolate
from glob import glob

#create xarray dataset with saildrone filenames
#data directory for saildrone data
data_dir = 'F:/data/cruise_data/saildrone/2020_atomic/'
saildrone_filename_short=['saildrone-gen_5-atomic_eurec4a_2020-sd1026-20200117T000000-20200302T235959-1_minutes-v1.1589306725934.nc',
                          'saildrone-gen_5-atomic_eurec4a_2020-sd1060-20200117T000000-20200302T235959-1_minutes-v1.1589306886594.nc',
                          'saildrone-gen_5-atomic_eurec4a_2020-sd1061-20200117T000000-20200302T235959-1_minutes-v1.1589307121602.nc'
                          
                         ]
saildrone_filename=[data_dir+'/delay_mode/1026/1min/'+saildrone_filename_short[0],
                    data_dir+'/delay_mode/1060/1min/'+saildrone_filename_short[1],
                    data_dir+'/delay_mode/1061/1min/'+saildrone_filename_short[2]
                   ]

#data direcgtory for temperature logger .csv files
adir_sbe='F:/data/cruise_data/saildrone/2020_atomic/temp_log_proc/'

#get list of all filenames in directory
files = [x for x in glob(adir_sbe+'*1_min*.nc')]
#print('number of file:',len(files))
#print(files)

usvs=[1026,1060,1061]
ds_info_saildrone = xr.Dataset(data_vars={'fname_usv':(['trajectory'],saildrone_filename),
                                          'fname_logger':(['trajectory'],files),
                                          'fname_short_usv':(['trajectory'],saildrone_filename_short)
                                         },coords={'trajectory':usvs})



print(ds_info_saildrone.sel(trajectory=1061).fname_logger.data)
print(ds_info_saildrone.sel(trajectory=1061).fname_usv.data)



In [ ]:
iusv=1061
#open both files
fname=str(ds_info_saildrone.sel(trajectory=iusv).fname_usv.data)
ds_usv=xr.open_dataset(fname)
ds_usv.close()
fname=str(ds_info_saildrone.sel(trajectory=iusv).fname_logger.data)
ds_logger=xr.open_dataset(fname)
ds_logger.close()
#swap dims to time since obs makes no sense
ds_usv = ds_usv.isel(trajectory=0).swap_dims({'obs':'time'})
ds_logger = ds_logger.isel(trajectory=0).swap_dims({'obs':'time'})

In [ ]:
ds_usv.TEMP_SBE37_MEAN.plot()
ds_logger.sea_water_temperature_00_mean.plot()

In [ ]:
ds_logger.time